In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import dlt
import duckdb
import os

In [2]:
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet"
file_name = "output.parquet"
os.system(f"wget {url} -O {file_name}")

--2024-02-17 16:32:31--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.161.108.141, 18.161.108.77, 18.161.108.231, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.161.108.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1254291 (1.2M) [binary/octet-stream]
Saving to: ‘output.parquet’

     0K .......... .......... .......... .......... ..........  4%  381K 3s
    50K .......... .......... .......... .......... ..........  8%  509K 3s
   100K .......... .......... .......... .......... .......... 12% 1.24M 2s
   150K .......... .......... .......... .......... .......... 16% 1.27M 2s
   200K .......... .......... .......... .......... .......... 20% 1.24M 1s
   250K .......... .......... .......... .......... .......... 24% 1.84M 1s
   300K .......... .......... .......... .......... .......... 28% 1.89M 1s
   350K

0

In [3]:
df = pa.parquet.read_pandas(file_name, columns=None)

In [6]:
parquet_file = pq.ParquetFile(file_name)

In [7]:
parquet_file.read()

pyarrow.Table
VendorID: int64
lpep_pickup_datetime: timestamp[us]
lpep_dropoff_datetime: timestamp[us]
store_and_fwd_flag: string
RatecodeID: double
PULocationID: int64
DOLocationID: int64
passenger_count: double
trip_distance: double
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
ehail_fee: null
improvement_surcharge: double
total_amount: double
payment_type: double
trip_type: double
congestion_surcharge: double
----
VendorID: [[2,1,1,2,2,...,2,2,2,2,2]]
lpep_pickup_datetime: [[2022-01-01 00:14:21.000000,2022-01-01 00:20:55.000000,2022-01-01 00:57:02.000000,2022-01-01 00:07:42.000000,2022-01-01 00:07:50.000000,...,2022-01-31 23:25:00.000000,2022-01-31 23:52:00.000000,2022-01-31 23:17:00.000000,2022-01-31 23:45:00.000000,2022-01-31 23:52:00.000000]]
lpep_dropoff_datetime: [[2022-01-01 00:15:33.000000,2022-01-01 00:29:38.000000,2022-01-01 01:13:14.000000,2022-01-01 00:15:57.000000,2022-01-01 00:28:52.000000,...,2022-01-31 23:33:00.000000,2022-0

In [10]:
pipeline = dlt.pipeline(pipeline_name="taxi_data",
						destination='duckdb', 
						dataset_name='taxi_rides')

In [14]:
info = pipeline.run(data, 
                    table_name="users", 
                    write_disposition="replace")

NameError: name 'data' is not defined

In [15]:
info = pipeline.run(url, loader_file_format="parquet")

PipelineStepFailed: Pipeline execution failed at stage extract with exception:

<class 'dlt.extract.exceptions.ResourceNameMissing'>
Resource name is missing. If you create a resource directly from data ie. from a list you must pass the name explicitly in `name` argument.
        Please note that for resources created from functions or generators, the name is the function name by default.

In [63]:
import dlt
import pandas as pd

owid_disasters_csv = (
    "https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv"
)
df = pd.read_csv(owid_disasters_csv)
data = df.to_dict(orient="records")

pipeline = dlt.pipeline(
    pipeline_name='pipeline',
    #destination='dlt.destinations.filesystem'
    destination='bigquery',
    staging='filesystem', # add this to activate the staging location
    dataset_name='mydata'
)

#load_info = pipeline.run(data)
#load_info = pipeline.run(data, table_name="zone_lookup")
load_info = pipeline.run(data, table_name="zone_lookup")

print(load_info)

PipelineStepFailed: Pipeline execution failed at stage sync with exception:

<class 'dlt.common.configuration.exceptions.ConfigFieldMissingException'>
Following fields are missing: ['bucket_url'] in configuration with spec FilesystemDestinationClientConfiguration
	for field "bucket_url" config providers and keys were tried in following order:
		In Environment Variables key PIPELINE__DESTINATION__FILESYSTEM__BUCKET_URL was not found.
		In Environment Variables key PIPELINE__DESTINATION__BUCKET_URL was not found.
		In Environment Variables key PIPELINE__BUCKET_URL was not found.
		In secrets.toml key pipeline.destination.filesystem.bucket_url was not found.
		In secrets.toml key pipeline.destination.bucket_url was not found.
		In secrets.toml key pipeline.bucket_url was not found.
		In Environment Variables key DESTINATION__FILESYSTEM__BUCKET_URL was not found.
		In Environment Variables key DESTINATION__BUCKET_URL was not found.
		In Environment Variables key BUCKET_URL was not found.
	

In [58]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

In [59]:
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

In [60]:
display(conn.sql("DESCRIBE"))

┌──────────┬─────────┬─────────────────────┬──────────────────────┬────────────────────────────────────────┬───────────┐
│ database │ schema  │        name         │     column_names     │              column_types              │ temporary │
│ varchar  │ varchar │       varchar       │      varchar[]       │               varchar[]                │  boolean  │
├──────────┼─────────┼─────────────────────┼──────────────────────┼────────────────────────────────────────┼───────────┤
│ pipeline │ mydata  │ _dlt_loads          │ [load_id, schema_n…  │ [VARCHAR, VARCHAR, BIGINT, TIMESTAMP…  │ false     │
│ pipeline │ mydata  │ _dlt_pipeline_state │ [version, engine_v…  │ [BIGINT, BIGINT, VARCHAR, VARCHAR, T…  │ false     │
│ pipeline │ mydata  │ _dlt_version        │ [version, engine_v…  │ [BIGINT, BIGINT, TIMESTAMP WITH TIME…  │ false     │
│ pipeline │ mydata  │ zone_lookup         │ [location_id, boro…  │ [BIGINT, VARCHAR, VARCHAR, VARCHAR, …  │ false     │
└──────────┴─────────┴──────────

In [61]:
stats_table = conn.sql("SELECT * FROM zone_lookup").df()
display(stats_table)

,location_id,borough,zone,service_zone,_dlt_load_id,_dlt_id
0,1,EWR,Newark Airport,EWR,1708195132.3040164,2fl3giQ/bQWXXg
1,2,Queens,Jamaica Bay,Boro Zone,1708195132.3040164,5yn65bCL+3HUoQ
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,1708195132.3040164,hFXu8PoyWgz9FA
3,4,Manhattan,Alphabet City,Yellow Zone,1708195132.3040164,dRUGHtcs/Yb2uQ
4,5,Staten Island,Arden Heights,Boro Zone,1708195132.3040164,IohlN9Qh82hZ4w
...,...,...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone,1708195132.3040164,PyyGcG0hZHHkwg
261,262,Manhattan,Yorkville East,Yellow Zone,1708195132.3040164,0dqCCTG2ouSAbA
262,263,Manhattan,Yorkville West,Yellow Zone,1708195132.3040164,WNtzLun9PNJvtw
263,264,Unknown,NV,None,1708195132.3040164,BIyoDfUPFHV+iQ
